# Plot Contribution Predictions

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from glob import glob
from tqdm import tqdm

In [ ]:
player_info = pd.read_csv('data/raw_pbp/playerlist.csv', index_col=[0])

In [ ]:
pred_files = sorted(glob('data/contribution_predictions/*.pkl'))

In [ ]:
for pred_file in tqdm(pred_files):
    
    df = pd.read_pickle(pred_file)
    
    df['weight'] = 1 / df.cv_mse
    
    player_off = df.groupby('PERSON_ID').apply(lambda x: (x['weight'] * x['off_contribution_norm']).sum() / x['weight'].sum()).reset_index()
    player_off.rename(columns={0:'off'}, inplace=True)

    player_def = df.groupby('PERSON_ID').apply(lambda x: (x['weight'] * x['def_contribution_norm']).sum() / x['weight'].sum()).reset_index()
    player_def.rename(columns={0:'def'}, inplace=True)

    player_time = df.groupby('PERSON_ID').TIME.min().reset_index()
    player_time.rename(columns={0:'time'}, inplace=True)
    
    res = pd.merge(player_off, player_def, on='PERSON_ID').merge(player_time, on='PERSON_ID')
    res = res.merge(player_info[['PERSON_ID', 'DISPLAY_FIRST_LAST']], on='PERSON_ID')
    
    fig = px.scatter(res, x="off", y="def", hover_name='DISPLAY_FIRST_LAST', size='TIME')

    fig.add_vline(x=0, line_width=2, line_dash="dash", line_color="red")
    fig.add_hline(y=0, line_width=2, line_dash="dash", line_color="red")

    #fig.show()

    pio.write_html(fig, file='plots/' + pred_file.split('/')[-1].replace('.pkl', '.html'), auto_open=False)